# TensorBoard使用

In [1]:
import tensorflow as tf
import time
import os

In [2]:
tf.__version__

'2.0.0'

In [3]:
# 读取数据集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
train_num = x_train.shape[0]
test_num = x_test.shape[0]
print("x_train.shape:", x_train.shape)
print("x_test.shape", x_test.shape)

x_train.shape: (60000, 28, 28)
x_test.shape (10000, 28, 28)


In [4]:
# 维度扩增
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)
print("x_train.shape:", x_train.shape)
print("x_test.shape", x_test.shape)

x_train.shape: (60000, 28, 28, 1)
x_test.shape (10000, 28, 28, 1)


In [5]:
# 数据类型转换
x_train = tf.cast(x_train / 255, tf.float32)
y_train = tf.cast(y_train, tf.int64)

x_test = tf.cast(x_test / 255, tf.float32)
y_test = tf.cast(y_test, tf.int64)

In [6]:
# 创建数据集
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
print("train_dataset:", train_dataset)
print("test_dataset:", test_dataset)

train_dataset: <TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>
test_dataset: <TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [7]:
BATCH_SIZE = 64
# 用于自定义训练
train_dataset_customer = train_dataset.shuffle(train_num).batch(BATCH_SIZE)
test_dataset_customer = test_dataset.batch(BATCH_SIZE)
# 用于keras训练
train_dataset = train_dataset.shuffle(train_num).repeat().batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
print("train_dataset:", train_dataset)
print("test_dataset:", test_dataset)

train_dataset: <BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
test_dataset: <BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [8]:
# 构建模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
global_max_pooling2d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 在keras中使用tensorboard

In [10]:
time.strftime("%Y%m%d-%H_%M_%S", time.localtime())

'20191214-19_40_51'

In [11]:
# 获取当前时间
time_str = time.strftime("%Y%m%d-%H_%M_%S", time.localtime())
# 建立log目录
log_dir = os.path.join('E:\\log', time_str)

In [12]:
# 建立tensorboard回调函数
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
# tensorboard可视化自定义标量
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
# 控制lr变化函数
def lr_schedule(epoch):
    if epoch < 5:
        learning_rate = 0.005
    else:
        learning_rate = 0.001
    # 写入标量变化
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [14]:
model.fit(train_dataset,
          epochs=10, 
          steps_per_epoch = train_num // BATCH_SIZE, 
          validation_data = test_dataset, 
          validation_steps = test_num // BATCH_SIZE,
          callbacks=[tensorboard_callback, lr_callback])

Train for 937 steps, validate for 156 steps
Epoch 1/10
937/937 [==============================] - 8s 8ms/step - loss: 0.5515 - accuracy: 0.8247 - val_loss: 0.2533 - val_accuracy: 0.9239
Epoch 2/10
937/937 [==============================] - 6s 6ms/step - loss: 0.2585 - accuracy: 0.9179 - val_loss: 0.2012 - val_accuracy: 0.9346
Epoch 3/10
937/937 [==============================] - 6s 7ms/step - loss: 0.2256 - accuracy: 0.9284 - val_loss: 0.1823 - val_accuracy: 0.9455
Epoch 4/10
937/937 [==============================] - 6s 6ms/step - loss: 0.2059 - accuracy: 0.9358 - val_loss: 0.1752 - val_accuracy: 0.9453
Epoch 5/10
937/937 [==============================] - 6s 6ms/step - loss: 0.1888 - accuracy: 0.9394 - val_loss: 0.1822 - val_accuracy: 0.9463
Epoch 6/10
937/937 [==============================] - 6s 6ms/step - loss: 0.1514 - accuracy: 0.9522 - val_loss: 0.1507 - val_accuracy: 0.9527
Epoch 7/10
937/937 [==============================] - 6s 6ms/step - loss: 0.1464 - accuracy: 0.9542 - va


#### 在notebook中显示tensorboard
%load_ext tensorboard  
%matplotlib inline
#### 启动tensorboard
%tensorboard --logdir E:\\log
#### 浏览器中启动tensorboard，在命令行输入  
tensorboard --logdir E:\\log


### 自定义训练中使用Tensorboard

In [15]:
# 构建模型
model_customer = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model_customer.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 创建一个优化器实例
optimizer = tf.keras.optimizers.Adam()

In [17]:
# 创建loss函数的计算方式
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [18]:
# 创建评估训练的指标
train_loss = tf.keras.metrics.Mean("train_loss")
train_acc = tf.keras.metrics.SparseCategoricalAccuracy("train_acc")
test_loss = tf.keras.metrics.Mean("test_loss")
test_acc = tf.keras.metrics.SparseCategoricalAccuracy("test_acc")

In [19]:
# 训练一步
def train_one_step(model, x, y_true):
    with tf.GradientTape() as t:
        # 计算损失
        y_pred = model(x)
        loss = loss_func(y_true, y_pred)
    # 计算梯度
    gradients = t.gradient(loss, model.trainable_variables)
    # 更新参数
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # 计算训练集的评估指标
    train_loss(loss)
    train_acc(y_true, y_pred)

In [20]:
# 测试一步
def test_one_step(model, x, y_true):
    # 计算loss
    y_pred = model(x)
    loss = loss_func(y_true, y_pred)
    # 计算测试集的评估指标
    test_loss(loss)
    test_acc(y_true, y_pred)

In [21]:
# 定义log保存路径
current_time = time.strftime("%Y%m%d-%H_%M_%S", time.localtime())
train_log_dir = 'E:/log/gradient_tape/' + current_time + '/train'
test_log_dir = 'E:/log/gradient_tape/' + current_time + '/test'
# 定义writer
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [22]:
def train(model, train_dataset, test_dataset, epochs):
    for epoch in range(epochs):
        # 一次取出一个batch进行训练
        for (x, y_true) in train_dataset:
            train_one_step(model, x, y_true)
        # 写入训练数据
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_acc.result(), step=epoch)
        # 测试
        for (x, y_true) in test_dataset:
            test_one_step(model, x, y_true)
        # 写入测试数据
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_acc.result(), step=epoch)
            
        print("Epoch{} train_loss is {:.2f}, train_acc is {:.2f}, test_loss is {:.2f}, test_acc is {:.2f}".format(epoch+1, 
                                                                                                                  train_loss.result(),
                                                                                                                  train_acc.result(),
                                                                                                                  test_loss.result(),
                                                                                                                  test_acc.result()))
        train_loss.reset_states()
        train_acc.reset_states()
        test_loss.reset_states()
        test_acc.reset_states()

In [23]:
train(model_customer, train_dataset_customer, test_dataset_customer, epochs=10)

Epoch1 train_loss is 1.05, train_acc is 0.69, test_loss is 0.47, test_acc is 0.86
Epoch2 train_loss is 0.42, train_acc is 0.87, test_loss is 0.34, test_acc is 0.89
Epoch3 train_loss is 0.34, train_acc is 0.89, test_loss is 0.29, test_acc is 0.91
Epoch4 train_loss is 0.30, train_acc is 0.91, test_loss is 0.27, test_acc is 0.92
Epoch5 train_loss is 0.27, train_acc is 0.91, test_loss is 0.24, test_acc is 0.92
Epoch6 train_loss is 0.26, train_acc is 0.92, test_loss is 0.23, test_acc is 0.93
Epoch7 train_loss is 0.24, train_acc is 0.92, test_loss is 0.21, test_acc is 0.93
Epoch8 train_loss is 0.23, train_acc is 0.93, test_loss is 0.21, test_acc is 0.93
Epoch9 train_loss is 0.22, train_acc is 0.93, test_loss is 0.20, test_acc is 0.94
Epoch10 train_loss is 0.21, train_acc is 0.93, test_loss is 0.20, test_acc is 0.94
